In [47]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [48]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [49]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('CUDA version:', torch.version.cuda)
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce GTX 960
CUDA version: 11.7
Memory Usage:
Allocated: 0.1 GB
Cached:    2.9 GB


In [50]:
import numpy as np
import PIL
from PIL import Image
import datasets
import os
import evaluate
import torch
import json
import codecs

In [51]:
image_folder = './WE3DS/images/'
annotation_folder = './WE3DS/annotations/segmentation/SegmentationLabel/'
# Define the paths to the images and annotations
all_image_names = np.array(os.listdir(image_folder))

In [52]:
plant_classification = {
    'void': 'void',
    'soil': 'soil',
    'broad bean': 'crop',
    'corn spurry': 'weed',
    'red-root amaranth': 'weed',
    'common buckwheat': 'crop',
    'pea': 'crop',
    'red fingergrass': 'weed',
    'common wild oat': 'weed',
    'cornflower': 'weed',
    'corn cockle': 'weed',
    'corn': 'crop',
    'milk thistle': 'weed',
    'rye brome': 'weed',
    'soybean': 'crop',
    'sunflower': 'crop',
    'narrow-leaved plantain': 'weed',
    'small-flower geranium': 'weed',
    'sugar beet': 'crop'
}

In [53]:
def get_image_meta_filepath(plant_name):
    return './meta/' + plant_name + '_images.json'

In [54]:
def get_image_list_for_crop(crop_name):
    # Create an empty list to store the dataset
    image_list = []
    crop_image_names = json.load(codecs.open(get_image_meta_filepath(crop_name), 'r', 'utf-8-sig'))
    weed_image_names = json.load(codecs.open(get_image_meta_filepath('no_crop'), 'r', 'utf-8-sig'))
    image_names = crop_image_names + weed_image_names

    # Iterate over the image and annotation paths
    for image_name in crop_image_names:
        # Load the image and annotation using PIL
        image = Image.open(image_folder + image_name)
        annotation = Image.open(annotation_folder + image_name)
        
        # Create a dictionary entry for the dataseta
        entry = {'image': image, 'annotation': annotation}
        
        # Add the entry to the dataset
        image_list.append(entry)

    return image_list

In [55]:
def create_and_split_dataset_for_crop(crop_image_list):
    dataset = datasets.Dataset.from_list(crop_image_list)
    dataset = dataset.train_test_split(test_size=0.5)
    train_ds = dataset["train"]
    val_ds, test_ds = dataset["test"].train_test_split(test_size=0.5).values()
    return train_ds, val_ds, test_ds

In [56]:
broad_bean_image_list = get_image_list_for_crop('broad_bean')
broad_bean_train_ds, broad_bean_val_ds, broad_bean_test_ds = create_and_split_dataset_for_crop(broad_bean_image_list)

In [57]:
print("Training subset number of images: " + str(broad_bean_train_ds.num_rows))
print("Validation subset number of images: " + str(broad_bean_val_ds.num_rows))
print("Test subset number of images: " + str(broad_bean_test_ds.num_rows))

Training subset number of images: 105
Validation subset number of images: 52
Test subset number of images: 53


In [58]:
from transformers import AutoImageProcessor

checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, reduce_labels=True)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [59]:
def train_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs

In [60]:
metric = evaluate.load("mean_iou")

In [61]:
# with open('./WE3DS/class_names.txt', 'r') as file:
#     labels = [line.strip() for line in file]

# labels.remove('soil')
# ids = list(range(1, 20))
labels = ['void', 'broad_bean']
ids = list(range(1, 3))

id2label = dict(zip(ids, labels))
label2id = dict(zip(labels, ids))

num_labels = len(labels)

print("id2label:", id2label)
print("label2id:", label2id)

id2label: {1: 'void', 2: 'broad_bean'}
label2id: {'void': 1, 'broad_bean': 2}


In [62]:
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = torch.nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [63]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.3.proj.weight', 'decode_head.batch_norm.running_mean', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.classifier.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.batch_norm.bias', 'decode_head.linear_c.3.proj.bias', 'decode_head.batch_norm.weight', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.batch_norm.num_batches_tracked']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=100,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    seed=12,
)

In [65]:
def initialize_trainer(train_ds, test_ds) :
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    return trainer

In [66]:
broad_bean_train_ds.set_transform(train_transforms)
broad_bean_val_ds.set_transform(train_transforms)
broad_bean_test_ds.set_transform(train_transforms)

In [67]:
broad_bean_trainer = initialize_trainer(broad_bean_train_ds, broad_bean_val_ds)
broad_bean_trainer.train()

c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 1/2100 [00:04<2:29:52,  4.28s/it]

{'loss': 0.6935, 'learning_rate': 5.997142857142857e-05, 'epoch': 0.05}


  0%|          | 2/2100 [00:07<2:09:11,  3.69s/it]

{'loss': 0.676, 'learning_rate': 5.994285714285715e-05, 'epoch': 0.1}


  0%|          | 3/2100 [00:10<2:02:45,  3.51s/it]

...


 20%|██        | 420/2100 [50:55<1:29:28,  3.20s/it]

{'eval_loss': 0.03197423368692398, 'eval_mean_iou': 0.9481100681558784, 'eval_mean_accuracy': 0.9855434264433641, 'eval_overall_accuracy': 0.9978915017373339, 'eval_per_category_iou': [0.9978514571051735, 0.8983686792065833], 'eval_per_category_accuracy': [0.9983835634206012, 0.972703289466127], 'eval_runtime': 81.46, 'eval_samples_per_second': 0.638, 'eval_steps_per_second': 0.135, 'epoch': 20.0}


 20%|██        | 420/2100 [50:55<3:23:42,  7.28s/it]

{'train_runtime': 3055.5717, 'train_samples_per_second': 3.436, 'train_steps_per_second': 0.687, 'train_loss': 0.133953789098277, 'epoch': 20.0}


TrainOutput(global_step=420, training_loss=0.133953789098277, metrics={'train_runtime': 3055.5717, 'train_samples_per_second': 3.436, 'train_steps_per_second': 0.687, 'train_loss': 0.133953789098277, 'epoch': 20.0})

In [68]:
broad_bean_trainer.state.log_history

[{'loss': 0.6935,
  'learning_rate': 5.997142857142857e-05,
  'epoch': 0.05,
  'step': 1},
 {'loss': 0.676,
  'learning_rate': 5.994285714285715e-05,
  'epoch': 0.1,
  'step': 2},
 {'loss': 0.6713,
  'learning_rate': 5.9914285714285716e-05,
  'epoch': 0.14,
  'step': 3},
 {'loss': 0.6325,
  'learning_rate': 5.988571428571429e-05,
  'epoch': 0.19,
  'step': 4},
 {'loss': 0.6404,
  'learning_rate': 5.9857142857142856e-05,
  'epoch': 0.24,
  'step': 5},
 {'loss': 0.6362,
  'learning_rate': 5.982857142857143e-05,
  'epoch': 0.29,
  'step': 6},
 {'loss': 0.5927,
  'learning_rate': 5.9800000000000003e-05,
  'epoch': 0.33,
  'step': 7},
 {'loss': 0.5819,
  'learning_rate': 5.977142857142857e-05,
  'epoch': 0.38,
  'step': 8},
 {'loss': 0.6365,
  'learning_rate': 5.9742857142857144e-05,
  'epoch': 0.43,
  'step': 9},
 {'loss': 0.5707,
  'learning_rate': 5.971428571428572e-05,
  'epoch': 0.48,
  'step': 10},
 {'loss': 0.5955,
  'learning_rate': 5.9685714285714284e-05,
  'epoch': 0.52,
  'step':

In [69]:
# Save the trained model, so that it can be used for inference later.
# Save the log history, so that it can be used for plotting later.
broad_bean_trainer.save_model('models/broad_bean')
with open('models/broad_bean/broad_bean_log_history.json', 'w') as file:
    broad_bean_log_history = broad_bean_trainer.state.log_history
    json.dump(broad_bean_log_history, file)

In [101]:
outputs = broad_bean_trainer.predict(broad_bean_test_ds)

100%|██████████| 11/11 [01:13<00:00,  6.72s/it]


In [102]:
test_metric = broad_bean_trainer.evaluate(broad_bean_test_ds)

100%|██████████| 11/11 [01:18<00:00,  7.13s/it]


In [103]:
test_metric

{'eval_loss': 0.027685143053531647,
 'eval_mean_iou': 0.9514935731094303,
 'eval_mean_accuracy': 0.9739695790575061,
 'eval_overall_accuracy': 0.997216919396627,
 'eval_per_category_iou': [0.9971403494393721, 0.9058467967794884],
 'eval_per_category_accuracy': [0.99860670276049, 0.9493324553545223],
 'eval_runtime': 80.4715,
 'eval_samples_per_second': 0.659,
 'eval_steps_per_second': 0.137,
 'epoch': 20.0}

In [99]:
image = Image.open('./WE3DS/annotations/segmentation/SegmentationLabel/img_00000.png')
image.size[::-1]

(1144, 1600)

In [100]:
# upsampled_ouputs = torch.nn.functional.interpolate(
#     outputs,
#     size=image.size[::-1],
#     mode="bilinear",
#     align_corners=False,
# )
# y_pred = upsampled_ouputs.predictions.argmax(1)

AttributeError: 'PredictionOutput' object has no attribute 'dim'

In [ ]:
print("Predicting following unique classes: ", np.unique(y_pred))
print(y_pred.shape)

Predicting following unique classes:  [0 1]
(53, 128, 128)


In [ ]:
y_pred

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [94]:
broad_bean_trainer.optimizer

AcceleratedOptimizer (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-08
    initial_lr: 6e-05
    lr: 4.8e-05
    weight_decay: 0.0

Parameter Group 1
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-08
    initial_lr: 6e-05
    lr: 4.8e-05
    weight_decay: 0.0
)